In [3]:
# Establishing a connection to Snowflake through the Snowflake Connector, and writing the cui_map to Snowflake

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import requests

def snowflake_connection():
    """
    Establishes a connection to Snowflake used for later reading and writing of tables
    
    Parameters:
    -----------
    none
    
    Returns:
    --------
    conn: The connection to Snowflake
    
    
    """
    load_dotenv()

    conn = snowflake.connector.connect(
        user=os.getenv("SNOWFLAKE_USER"),
        password = os.getenv("SNOWFLAKE_PWD"),
        account = 'jl41563.us-east-2.aws',
        role='GROUSE_ROLE_C_ANALYTICS',
        warehouse = 'GROUSE_WH',
        database = 'GROUSE_DEID_ANALYTICS_DB',
        schema = 'GROUSE_SEMA')
    
    return conn


def write_to_snowflake(conn, df, name):
    '''
    Parameters:
    -----------
    conn: The snowflake connection
    df: DataFrame
    name: The name of the table to write to in Snowflake
    
    Returns:
    --------
    success, nchincks, nrows, output: Information about the table in Snowflake
    
    '''
    success, nchuncks, nrows, output = write_pandas(conn, df, name)
    return success, nchuncks, nrows, output

def read_ingredients(conn, sql = "SELECT * FROM INGREDIENT_LEVEL"):
    '''
    Reads the prescriptions_data table from snowflake into a DataFrame
    
    Parameters:
    -----------
    
    
    Returns:
    -------
    df_prescriptions: Nx3 DataFrame 
                        Column 1: RXCUI
                        Column 2: NAME'''
    
    cur = conn.cursor()
    df_ingredients = cur.execute(sql).fetch_pandas_all()
    return df_ingredients


conn = snowflake_connection()
df_ingredients = read_ingredients(conn)
df_ingredients.head()

def get_rxcuis_from_json(data):
    '''
    Extract list of rxcuis from json data
    Parameters:
    -----------
    data: json() object that is the output of the RXNORM getDrugs API
    
    Returns:
    --------
    rxcuis: a list of the all the rxcuis (as characters) associated with that drug name
    '''
    
    if('conceptGroup' not in data['drugGroup']):
        return None
    rxcuis = []
    concept_group = data['drugGroup']['conceptGroup']
    for i in range(len(concept_group)):
        drug_tty = concept_group[i]
        if('conceptProperties' in drug_tty):
            drug_list = drug_tty['conceptProperties']
            for j in range(len(drug_list)):
                rxcui = drug_list[j]['rxcui']
                rxcuis.append(rxcui)
    return rxcuis


def write_cui_map(df_ingredients):
    '''
    Write the dataframe with rxcuis from a list of drug names
    
    Parameters:
    -----------
    drug_names: a list of drug nonproprietary names
    
    Returns:
    --------
    df_cui_map: A Nx3 DataFrame
                    Column 1: INGREDIENT_NAME
                    Column 2: RXCUI_IN
                    Column 3: RXCUI_LOWER
    '''
    drug_names = df_ingredients["NAME"].tolist()
    ingredient_cuis = df_ingredients['RXCUI'].tolist()
    df_cui_map = pd.DataFrame(columns=['INGREDIENT_NAME', 'RXCUI_IN', 'RXCUI_LOWER'])
    cui_count = 0
    for i in range(len(drug_names)):
        if(drug_names[i] != None and type(drug_names[i]) == str):
            rxcui_in = ingredient_cuis[i]
            in_name = drug_names[i]
            URL = 'https://rxnav.nlm.nih.gov/REST/drugs.json?name=' + drug_names[i]
            response = requests.get(url=URL)
            data = response.json()

            #Getting the rxcuis from the drug
            if get_rxcuis_from_json(data) != None:
                rxcuis = get_rxcuis_from_json(data)
                cui_info = pd.DataFrame({'INGREDIENT_NAME':[in_name for j in range(len(rxcuis))], 
                                         'RXCUI_IN':[rxcui_in for k in range(len(rxcuis))], 
                                         'RXCUI_LOWER':rxcuis})
                df_cui_map = df_cui_map.append(cui_info, ignore_index=True)
    return df_cui_map



df_cui_map = write_cui_map(df_ingredients)

df_cui_map.head()

,INGREDIENT_NAME,RXCUI_IN,RXCUI_LOWER
0,xylazine,1099660,1099667
1,xylazine,1099660,1099901
2,xylazine,1099660,1536495
3,xylazine,1099660,1543402
4,xylazine,1099660,1099663


In [10]:
# Writing the df_cui_ids to the cui_ids table in snowflake
success, nchuncks, nrows, output = write_to_snowflake(conn, df_cui_map, "RXCUIS_MAP")

In [6]:
df_cui_map.to_csv('df_cui_map5.0', index=False)

In [4]:
'''Code to write the CUI_MAP to Snowflake from an existing .csv file'''
df_cui_map = pd.read_csv('df_cui_map')
print(df_cui_map.shape)
# Writing the df_cui_ids to the cui_ids table in snowflake
success, nchuncks, nrows, output = write_to_snowflake(conn, df_cui_map, "RXCUIS_MAP")

(39361, 3)


In [6]:
print(3)

3


In [2]:
# Install needed packages
!pip install snowflake-connector-python==2.7.7

     |████████████████████████████████| 16.5 MB 11.8 MB/s            
     |████████████████████████████████| 2.3 MB 68.8 MB/s            
     |████████████████████████████████| 194 kB 78.9 MB/s            
     |████████████████████████████████| 105 kB 83.3 MB/s            
  Attempting uninstall: asn1crypto
    Found existing installation: asn1crypto 1.4.0
    Uninstalling asn1crypto-1.4.0:
      Successfully uninstalled asn1crypto-1.4.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_latest_p37/bin/python -m pip install --upgrade pip' command.
